In [1]:
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import os 
import pandas as pd
import numpy as np
import random
import pickle
import pandas as pd
from Bio import SeqIO
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import roc_curve

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import load_model
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import imblearn
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, classification_report

2024-09-13 00:59:16.403067: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
os.chdir("/homes/t326h379/OGalNAc/Ankh_154_Proteins")

Training_Protein = []
with open("One_hundred_Thirty_Seven_Training_Proteins_OGalNAc.txt") as Subash:
    for line in Subash:
        Training_Protein.append(line.strip("\n"))
        
os.chdir("/home/t326h379/OGalNAc")

df_positive_positive = pd.read_csv('Positive_OGalNAc_ProtT5_Positive_Features_vectors_1837_Net_O_Glyc_4.txt',header=None)              
df_negative_negative = pd.read_csv('Negative_OGalNAc_ProtT5_Negative_Features_vectors_59414_Net_O_Glyc_4.txt',header=None)  

Header_name = ["Position","PID","S_or_T"]+[i for i in range(1,1025)]
df_positive_positive.columns = Header_name
df_negative_negative.columns = Header_name

df_positive = df_positive_positive[df_positive_positive['PID'].isin(Training_Protein)]
df_negative = df_negative_negative[df_negative_negative['PID'].isin(Training_Protein)]

# Multi Layer Perceptron 10 Fold CV

In [8]:
from imblearn.under_sampling import RandomUnderSampler
a = random.sample(range(1, 1000000), 20)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Positive_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        model = Sequential()

        model.add(Input(shape=(1024,)))

        model.add(Dense(64,activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(2,activation='softmax'))



        model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])

        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                        monitor = 'val_accuracy',
                                        verbose=0, 
                                        save_weights_only=False,
                                        save_best_only=True)

        reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=5, verbose=1, min_delta=1e-4, mode='max')

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

        history = model.fit(X_train, y_train_1, epochs=21,verbose=1, batch_size=32)

        Y_pred = model.predict(xval)
        Y_pred = (Y_pred > 0.5)
        y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
        y_pred = np.array(y_pred)

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("MLP Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(532, 1024) (46, 1024)
(14780, 1024) (948, 1024)
Training Dataset
(1064, 1024) (1064,)
Validation Dataset
(92, 1024) (92,)




Epoch 1/21
34/34 [==============================] - 1s 5ms/step - loss: 0.6012 - accuracy: 0.6889
Epoch 2/21
34/34 [==============================] - 0s 4ms/step - loss: 0.4751 - accuracy: 0.7820
Epoch 3/21
34/34 [==============================] - 0s 4ms/step - loss: 0.4057 - accuracy: 0.8158
Epoch 4/21
34/34 [==============================] - 0s 5ms/step - loss: 0.3633 - accuracy: 0.8553
Epoch 5/21
34/34 [==============================] - 0s 4ms/step - loss: 0.3185 - accuracy: 0.8778
Epoch 6/21
34/34 [==============================] - 0s 3ms/step - loss: 0.2910 - accuracy: 0.8957
Epoch 7/21
34/34 [==============================] - 0s 3ms/step - loss: 0.2605 - accuracy: 0.9117
Epoch 8/21
34/34 [==============================] - 0s 4ms/step - loss: 0.2335 - accuracy: 0.9239
Epoch 9/21
34/34 [==============================] - 0s 4ms/step - loss: 0.2143 - accuracy

(541, 1024) (37, 1024)
(14469, 1024) (1259, 1024)
Training Dataset
(1082, 1024) (1082,)
Validation Dataset
(74, 1024) (74,)




Epoch 1/21
34/34 [==============================] - 1s 2ms/step - loss: 0.5878 - accuracy: 0.7126
Epoch 2/21
34/34 [==============================] - 0s 2ms/step - loss: 0.4605 - accuracy: 0.7902
Epoch 3/21
34/34 [==============================] - 0s 2ms/step - loss: 0.4007 - accuracy: 0.8309
Epoch 4/21
34/34 [==============================] - 0s 2ms/step - loss: 0.3530 - accuracy: 0.8558
Epoch 5/21
34/34 [==============================] - 0s 2ms/step - loss: 0.3119 - accuracy: 0.8715
Epoch 6/21
34/34 [==============================] - 0s 2ms/step - loss: 0.2885 - accuracy: 0.8826
Epoch 7/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2559 - accuracy: 0.9076
Epoch 8/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2310 - accuracy: 0.9251
Epoch 9/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2168 - accurac

Epoch 5/21
34/34 [==============================] - 0s 1ms/step - loss: 0.3160 - accuracy: 0.8884
Epoch 6/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2809 - accuracy: 0.8987
Epoch 7/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2602 - accuracy: 0.9156
Epoch 8/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2253 - accuracy: 0.9306
Epoch 9/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2098 - accuracy: 0.9353
Epoch 10/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1875 - accuracy: 0.9437
Epoch 11/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1698 - accuracy: 0.9522
Epoch 12/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1514 - accuracy: 0.9540
Epoch 13/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1336 - accuracy: 0.9690
Epoch 14/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1190 - accuracy: 0.9747
Epoch 15/21
34/

33/33 [==============================] - 0s 4ms/step - loss: 0.2479 - accuracy: 0.9151
Epoch 9/21
33/33 [==============================] - 0s 4ms/step - loss: 0.2278 - accuracy: 0.9160
Epoch 10/21
33/33 [==============================] - 0s 5ms/step - loss: 0.2129 - accuracy: 0.9208
Epoch 11/21
33/33 [==============================] - 0s 3ms/step - loss: 0.1862 - accuracy: 0.9437
Epoch 12/21
33/33 [==============================] - 0s 2ms/step - loss: 0.1720 - accuracy: 0.9485
Epoch 13/21
33/33 [==============================] - 0s 2ms/step - loss: 0.1601 - accuracy: 0.9580
Epoch 14/21
33/33 [==============================] - 0s 2ms/step - loss: 0.1398 - accuracy: 0.9542
Epoch 15/21
33/33 [==============================] - 0s 2ms/step - loss: 0.1251 - accuracy: 0.9656
Epoch 16/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1156 - accuracy: 0.9723
Epoch 17/21
33/33 [==============================] - 0s 2ms/step - loss: 0.1144 - accuracy: 0.9695
Epoch 18/21
33/33 [====

34/34 [==============================] - 0s 1ms/step - loss: 0.2915 - accuracy: 0.9015
Epoch 7/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2526 - accuracy: 0.9182
Epoch 8/21
34/34 [==============================] - 0s 2ms/step - loss: 0.2310 - accuracy: 0.9210
Epoch 9/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2103 - accuracy: 0.9442
Epoch 10/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1921 - accuracy: 0.9470
Epoch 11/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1745 - accuracy: 0.9507
Epoch 12/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1572 - accuracy: 0.9554
Epoch 13/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1366 - accuracy: 0.9638
Epoch 14/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1324 - accuracy: 0.9684
Epoch 15/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1228 - accuracy: 0.9740
Epoch 16/21
34/34 [======

...

(528, 1024) (48, 1024)
(13802, 1024) (1825, 1024)
Training Dataset
(1056, 1024) (1056,)
Validation Dataset
(96, 1024) (96,)




Epoch 1/21
33/33 [==============================] - 1s 1ms/step - loss: 0.5970 - accuracy: 0.6932
Epoch 2/21
33/33 [==============================] - 0s 1ms/step - loss: 0.4845 - accuracy: 0.7812
Epoch 3/21
33/33 [==============================] - 0s 1ms/step - loss: 0.4287 - accuracy: 0.8087
Epoch 4/21
33/33 [==============================] - 0s 1ms/step - loss: 0.3849 - accuracy: 0.8314
Epoch 5/21
33/33 [==============================] - 0s 1ms/step - loss: 0.3461 - accuracy: 0.8608
Epoch 6/21
33/33 [==============================] - 0s 1ms/step - loss: 0.3093 - accuracy: 0.8826
Epoch 7/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2846 - accuracy: 0.8958
Epoch 8/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2592 - accuracy: 0.9148
Epoch 9/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2289 - accurac

33/33 [==============================] - 0s 1ms/step - loss: 0.3415 - accuracy: 0.8690
Epoch 6/21
33/33 [==============================] - 0s 1ms/step - loss: 0.3175 - accuracy: 0.8834
Epoch 7/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2824 - accuracy: 0.8881
Epoch 8/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2634 - accuracy: 0.9063
Epoch 9/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2412 - accuracy: 0.9235
Epoch 10/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2236 - accuracy: 0.9264
Epoch 11/21
33/33 [==============================] - 0s 1ms/step - loss: 0.2101 - accuracy: 0.9312
Epoch 12/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1875 - accuracy: 0.9388
Epoch 13/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1655 - accuracy: 0.9493
Epoch 14/21
33/33 [==============================] - 0s 1ms/step - loss: 0.1557 - accuracy: 0.9551
Epoch 15/21
33/33 [=======

34/34 [==============================] - 0s 2ms/step - loss: 0.2248 - accuracy: 0.9255
Epoch 9/21
34/34 [==============================] - 0s 1ms/step - loss: 0.2007 - accuracy: 0.9330
Epoch 10/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1797 - accuracy: 0.9451
Epoch 11/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1715 - accuracy: 0.9544
Epoch 12/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1476 - accuracy: 0.9637
Epoch 13/21
34/34 [==============================] - 0s 2ms/step - loss: 0.1344 - accuracy: 0.9628
Epoch 14/21
34/34 [==============================] - 0s 2ms/step - loss: 0.1205 - accuracy: 0.9693
Epoch 15/21
34/34 [==============================] - 0s 2ms/step - loss: 0.1062 - accuracy: 0.9767
Epoch 16/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1056 - accuracy: 0.9749
Epoch 17/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0934 - accuracy: 0.9777
Epoch 18/21
34/34 [====

34/34 [==============================] - 0s 1ms/step - loss: 0.1579 - accuracy: 0.9436
Epoch 14/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1519 - accuracy: 0.9464
Epoch 15/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1353 - accuracy: 0.9624
Epoch 16/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1221 - accuracy: 0.9624
Epoch 17/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1182 - accuracy: 0.9680
Epoch 18/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0994 - accuracy: 0.9774
Epoch 19/21
34/34 [==============================] - 0s 1ms/step - loss: 0.1006 - accuracy: 0.9699
Epoch 20/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0926 - accuracy: 0.9746
Epoch 21/21
3/3 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.3102526139970115
Confusion Matrix : 
 [[28 11]
 [16 23]]
Accuracy on test set:    0.6538461538461539
Sensitivity

34/34 [==============================] - 0s 1ms/step - loss: 0.0945 - accuracy: 0.9794
Epoch 19/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0830 - accuracy: 0.9841
Epoch 20/21
34/34 [==============================] - 0s 1ms/step - loss: 0.0803 - accuracy: 0.9804
Epoch 21/21
3/3 [==============================] - 0s 965us/step
Independent Testing Matthews Correlation:  0.5007733956671915
Confusion Matrix : 
 [[26 10]
 [ 8 28]]
Accuracy on test set:    0.75
Sensitivity:    0.7777777777777778 	 Specificity:    0.7222222222222222
Precision  : 0.7368421052631579
Area Under Curve:    0.75
(72, 1024) (72,)
(508, 1024) (63, 1024)
(14377, 1024) (1152, 1024)
Training Dataset
(1016, 1024) (1016,)
Validation Dataset
(126, 1024) (126,)




Epoch 1/21
32/32 [==============================] - 1s 3ms/step - loss: 0.5967 - accuracy: 0.7274
Epoch 2/21
32/32 [==============================] - 0s 2ms/step - loss: 0.4749 - accuracy: 0.7923
Epoch 3/21
32/32 [===========================

# Logistic Regression 10 Fold CV

In [13]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
a = random.sample(range(1, 1000000), 3)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Positive_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = LogisticRegression(penalty='l2',solver='lbfgs',random_state=seed)

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("LR Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("LR Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(402, 1024) (67, 1024)
(13403, 1024) (1730, 1024)
Training Dataset
(804, 1024) (804,)
Validation Dataset
(134, 1024) (134,)




Matthews Correlation :  0.5524850114241866
Confusion Matrix : 
 [[53 14]
 [16 51]]
Accuracy on test set:    0.7761194029850746
Sensitivity:    0.7611940298507462 	 Specificity:    0.7910447761194029
Area Under Curve:    0.7761194029850746
(416, 1024) (53, 1024)
(12190, 1024) (2943, 1024)
Training Dataset
(832, 1024) (832,)
Validation Dataset
(106, 1024) (106,)




Matthews Correlation :  0.4906533814626582
Confusion Matrix : 
 [[39 14]
 [13 40]]
Accuracy on test set:    0.7452830188679245
Sensitivity:    0.7547169811320755 	 Specificity:    0.7358490566037735
Area Under Curve:    0.7452830188679247
(430, 1024) (39, 1024)
(13173, 1024) (1960, 1024)
Training Dataset
(860, 1024) (860,)
Validation Dataset
(78, 1024) (78,)




Matthews Correlation :  0.6932198240501807
Confusion Matrix : 
 [[34  5]
 [ 7 32]]
Accuracy on test set:    0.8461538461538461
Sensitivity: 

/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Matthews Correlation :  0.3890535739948358
Confusion Matrix : 
 [[32 17]
 [13 36]]
Accuracy on test set:    0.6938775510204082
Sensitivity:    0.7346938775510204 	 Specificity:    0.6530612244897959
Area Under Curve:    0.6938775510204082
(441, 1024) (28, 1024)
(14028, 1024) (1105, 1024)
Training Dataset
(882, 1024) (882,)
Validation Dataset
(56, 1024) (56,)




Matthews Correlation :  0.42857142857142855
Confusion Matrix : 
 [[20  8]
 [ 8 20]]
Accuracy on test set:    0.7142857142857143
Sensitivity:    0.7142857142857143 	 Specificity:    0.7142857142857143
Area Under Curve:    0.7142857142857143
(416, 1024) (53, 1024)
(13929, 1024) (1204, 1024)
Training Dataset
(832, 1024) (832,)
Validation Dataset
(106, 1024) (106,)




Matthews Correlation :  0.5127301186201267
Confusion Matrix : 
 [[43 10]
 [16 37]]
Accuracy on test set:    0.7547169811320755
Sensitivity:    0.6981132075471698 	 Specificity:    0.8113207547169812
Area Under Curve:    0.7547169811320755
LR Mean MCC = 0.5237 ± 0.085

/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Matthews Correlation :  0.48900964692182575
Confusion Matrix : 
 [[34 11]
 [12 33]]
Accuracy on test set:    0.7444444444444445
Sensitivity:    0.7333333333333333 	 Specificity:    0.7555555555555555
Area Under Curve:    0.7444444444444445
(504, 1024) (72, 1024)
(14496, 1024) (1195, 1024)
Training Dataset
(1008, 1024) (1008,)
Validation Dataset
(144, 1024) (144,)






...

/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Matthews Correlation :  0.7216671277871474
Confusion Matrix : 
 [[42  4]
 [ 9 37]]
Accuracy on test set:    0.8586956521739131
Sensitivity:    0.8043478260869565 	 Specificity:    0.9130434782608695
Area Under Curve:    0.8586956521739131
(528, 1024) (41, 1024)
(14876, 1024) (904, 1024)
Training Dataset
(1056, 1024) (1056,)
Validation Dataset
(82, 1024) (82,)






/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Matthews Correlation :  0.5869391856534222
Confusion Matrix : 
 [[31 10]
 [ 7 34]]
Accuracy on test set:    0.7926829268292683
Sensitivity:    0.8292682926829268 	 Specificity:    0.7560975609756098
Area Under Curve:    0.7926829268292683
(506, 1024) (63, 1024)
(14430, 1024) (1350, 1024)
Training Dataset
(1012, 1024) (1012,)
Validation Dataset
(126, 1024) (126,)




Matthews Correlation :  0.2733062457290543
Confusion Matrix : 
 [[45 18]
 [28 35]]
Accuracy on test set:    0.6349206349206349
Sensitivity:    0.5555555555555556 	 Specificity:    0.7142857142857143
Area Under Curve:    0.6349206349206349
LR Mean MCC = 0.4630 ± 0.1534 and Mean AUC = 0.7284 ± 0.0795
LR Mean MCC:    0.4629597384886531
Mean Sensitivity = 0.6798 ± 0.1591 and Mean Specificity = 0.7770 ± 0.0648 and Mean Accuracy = 0.7284 ± 0.0795
Mean Precision = 0.7489 ± 0.0657 
















/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Random Forest 10 Fold CV

In [10]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
a = random.sample(range(1, 1000000), 20)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Positive_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = RandomForestClassifier(max_depth=10, random_state=seed)

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("RF Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("RF Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(506, 1024) (66, 1024)
(13031, 1024) (2317, 1024)
Training Dataset
(1012, 1024) (1012,)
Validation Dataset
(132, 1024) (132,)




Matthews Correlation :  0.39036002917941326
Confusion Matrix : 
 [[57  9]
 [33 33]]
Accuracy on test set:    0.6818181818181818
Sensitivity:    0.5 	 Specificity:    0.8636363636363636
Area Under Curve:    0.6818181818181818
(429, 1024) (143, 1024)
(13298, 1024) (2050, 1024)
Training Dataset
(858, 1024) (858,)
Validation Dataset
(286, 1024) (286,)




Matthews Correlation :  0.5117423088555023
Confusion Matrix : 
 [[103  40]
 [ 30 113]]
Accuracy on test set:    0.7552447552447552
Sensitivity:    0.7902097902097902 	 Specificity:    0.7202797202797203
Area Under Curve:    0.7552447552447552
(515, 1024) (57, 1024)
(13987, 1024) (1361, 1024)
Training Dataset
(1030, 1024) (1030,)
Validation Dataset
(114, 1024) (114,)




Matthews Correlation :  0.5270462766947299
Confusion Matrix : 
 [[45 12]
 [15 42]]
Accuracy on test set:    0.7631578947368421
Sensitivity:    

(532, 1024) (45, 1024)
(11816, 1024) (3675, 1024)
Training Dataset
(1064, 1024) (1064,)
Validation Dataset
(90, 1024) (90,)




Matthews Correlation :  0.4000988020269483
Confusion Matrix : 
 [[32 13]
 [14 31]]
Accuracy on test set:    0.7
Sensitivity:    0.6888888888888889 	 Specificity:    0.7111111111111111
Area Under Curve:    0.7
(531, 1024) (46, 1024)
(13681, 1024) (1810, 1024)
Training Dataset
(1062, 1024) (1062,)
Validation Dataset
(92, 1024) (92,)




Matthews Correlation :  0.6086956521739131
Confusion Matrix : 
 [[37  9]
 [ 9 37]]
Accuracy on test set:    0.8043478260869565
Sensitivity:    0.8043478260869565 	 Specificity:    0.8043478260869565
Area Under Curve:    0.8043478260869565
(527, 1024) (50, 1024)
(13898, 1024) (1593, 1024)
Training Dataset
(1054, 1024) (1054,)
Validation Dataset
(100, 1024) (100,)




Matthews Correlation :  0.375
Confusion Matrix : 
 [[27 23]
 [ 9 41]]
Accuracy on test set:    0.68
Sensitivity:    0.82 	 Specificity:    0.54
Area Under Curve:    0

(413, 1024) (58, 1024)
(12010, 1024) (3018, 1024)
Training Dataset
(826, 1024) (826,)
Validation Dataset
(116, 1024) (116,)




Matthews Correlation :  0.48741743667595394
Confusion Matrix : 
 [[47 11]
 [19 39]]
Accuracy on test set:    0.7413793103448276
Sensitivity:    0.6724137931034483 	 Specificity:    0.8103448275862069
Area Under Curve:    0.7413793103448276
(409, 1024) (62, 1024)
(13631, 1024) (1397, 1024)
Training Dataset
(818, 1024) (818,)
Validation Dataset
(124, 1024) (124,)




Matthews Correlation :  0.5987243137499596
Confusion Matrix : 
 [[47 15]
 [10 52]]
Accuracy on test set:    0.7983870967741935
Sensitivity:    0.8387096774193549 	 Specificity:    0.7580645161290323
Area Under Curve:    0.7983870967741935
RF Mean MCC = 0.4980 ± 0.0932 and Mean AUC = 0.7479 ± 0.0465
RF Mean MCC:    0.49803756424033035
Mean Sensitivity = 0.7387 ± 0.0862 and Mean Specificity = 0.7571 ± 0.0379 and Mean Accuracy = 0.7479 ± 0.0465
Mean Precision = 0.7518 ± 0.0368 














(535, 1024

Matthews Correlation :  0.3227137185558497
Confusion Matrix : 
 [[46 10]
 [29 27]]
Accuracy on test set:    0.6517857142857143
Sensitivity:    0.48214285714285715 	 Specificity:    0.8214285714285714
Area Under Curve:    0.6517857142857143
(518, 1024) (59, 1024)
(13748, 1024) (1899, 1024)
Training Dataset
(1036, 1024) (1036,)
Validation Dataset
(118, 1024) (118,)




Matthews Correlation :  0.5281619138643888
Confusion Matrix : 
 [[42 17]
 [11 48]]
Accuracy on test set:    0.7627118644067796
Sensitivity:    0.8135593220338984 	 Specificity:    0.711864406779661
Area Under Curve:    0.7627118644067796
(541, 1024) (36, 1024)
(13998, 1024) (1649, 1024)
Training Dataset
(1082, 1024) (1082,)
Validation Dataset
(72, 1024) (72,)




Matthews Correlation :  0.4724045122202191
Confusion Matrix : 
 [[26 10]
 [ 9 27]]
Accuracy on test set:    0.7361111111111112
Sensitivity:    0.75 	 Specificity:    0.7222222222222222
Area Under Curve:    0.736111111111111
(422, 1024) (155, 1024)
(13680, 1024) (1

(517, 1024) (59, 1024)
(13776, 1024) (1880, 1024)
Training Dataset
(1034, 1024) (1034,)
Validation Dataset
(118, 1024) (118,)




Matthews Correlation :  0.49069974606294714
Confusion Matrix : 
 [[51  8]
 [23 36]]
Accuracy on test set:    0.7372881355932204
Sensitivity:    0.6101694915254238 	 Specificity:    0.864406779661017
Area Under Curve:    0.7372881355932204
(532, 1024) (44, 1024)
(14173, 1024) (1483, 1024)
Training Dataset
(1064, 1024) (1064,)
Validation Dataset
(88, 1024) (88,)




Matthews Correlation :  0.642364054837573
Confusion Matrix : 
 [[39  5]
 [11 33]]
Accuracy on test set:    0.8181818181818182
Sensitivity:    0.75 	 Specificity:    0.8863636363636364
Area Under Curve:    0.8181818181818181
(536, 1024) (40, 1024)
(14652, 1024) (1004, 1024)
Training Dataset
(1072, 1024) (1072,)
Validation Dataset
(80, 1024) (80,)




Matthews Correlation :  0.5332285519249769
Confusion Matrix : 
 [[27 13]
 [ 6 34]]
Accuracy on test set:    0.7625
Sensitivity:    0.85 	 Specificity: 

Matthews Correlation :  0.3315938128703508
Confusion Matrix : 
 [[33  8]
 [20 21]]
Accuracy on test set:    0.6585365853658537
Sensitivity:    0.5121951219512195 	 Specificity:    0.8048780487804879
Area Under Curve:    0.6585365853658537
(403, 1024) (151, 1024)
(14412, 1024) (1150, 1024)
Training Dataset
(806, 1024) (806,)
Validation Dataset
(302, 1024) (302,)




Matthews Correlation :  0.17227990392615053
Confusion Matrix : 
 [[91 60]
 [65 86]]
Accuracy on test set:    0.5860927152317881
Sensitivity:    0.5695364238410596 	 Specificity:    0.6026490066225165
Area Under Curve:    0.5860927152317881
RF Mean MCC = 0.4502 ± 0.1302 and Mean AUC = 0.7192 ± 0.0640
RF Mean MCC:    0.45024262741305254
Mean Sensitivity = 0.6780 ± 0.1316 and Mean Specificity = 0.7604 ± 0.1106 and Mean Accuracy = 0.7192 ± 0.0640
Mean Precision = 0.7503 ± 0.0898 














(520, 1024) (50, 1024)
(13558, 1024) (2192, 1024)
Training Dataset
(1040, 1024) (1040,)
Validation Dataset
(100, 1024) (100,)




Matthew

Matthews Correlation :  0.6601320396132047
Confusion Matrix : 
 [[41  9]
 [ 8 42]]
Accuracy on test set:    0.83
Sensitivity:    0.84 	 Specificity:    0.82
Area Under Curve:    0.83
(495, 1024) (82, 1024)
(12904, 1024) (2612, 1024)
Training Dataset
(990, 1024) (990,)
Validation Dataset
(164, 1024) (164,)




Matthews Correlation :  0.5507910552258941
Confusion Matrix : 
 [[67 15]
 [22 60]]
Accuracy on test set:    0.774390243902439
Sensitivity:    0.7317073170731707 	 Specificity:    0.8170731707317073
Area Under Curve:    0.774390243902439
(539, 1024) (38, 1024)
(14067, 1024) (1449, 1024)
Training Dataset
(1078, 1024) (1078,)
Validation Dataset
(76, 1024) (76,)




Matthews Correlation :  0.4763305116224668
Confusion Matrix : 
 [[30  8]
 [12 26]]
Accuracy on test set:    0.7368421052631579
Sensitivity:    0.6842105263157895 	 Specificity:    0.7894736842105263
Area Under Curve:    0.7368421052631579
(533, 1024) (44, 1024)
(14305, 1024) (1211, 1024)
Training Dataset
(1066, 1024) (1066

(415, 1024) (160, 1024)
(11528, 1024) (4248, 1024)
Training Dataset
(830, 1024) (830,)
Validation Dataset
(320, 1024) (320,)




Matthews Correlation :  0.5187601321327783
Confusion Matrix : 
 [[121  39]
 [ 38 122]]
Accuracy on test set:    0.759375
Sensitivity:    0.7625 	 Specificity:    0.75625
Area Under Curve:    0.7593749999999999
(520, 1024) (55, 1024)
(14324, 1024) (1452, 1024)
Training Dataset
(1040, 1024) (1040,)
Validation Dataset
(110, 1024) (110,)




Matthews Correlation :  0.49923017660270624
Confusion Matrix : 
 [[46  9]
 [19 36]]
Accuracy on test set:    0.7454545454545455
Sensitivity:    0.6545454545454545 	 Specificity:    0.8363636363636363
Area Under Curve:    0.7454545454545455
(503, 1024) (72, 1024)
(14060, 1024) (1716, 1024)
Training Dataset
(1006, 1024) (1006,)
Validation Dataset
(144, 1024) (144,)




Matthews Correlation :  0.40375250883334796
Confusion Matrix : 
 [[48 24]
 [19 53]]
Accuracy on test set:    0.7013888888888888
Sensitivity:    0.736111111111111

(522, 1024) (57, 1024)
(14349, 1024) (1241, 1024)
Training Dataset
(1044, 1024) (1044,)
Validation Dataset
(114, 1024) (114,)




Matthews Correlation :  0.40450601477770304
Confusion Matrix : 
 [[38 19]
 [15 42]]
Accuracy on test set:    0.7017543859649122
Sensitivity:    0.7368421052631579 	 Specificity:    0.6666666666666666
Area Under Curve:    0.7017543859649125
(518, 1024) (61, 1024)
(14058, 1024) (1532, 1024)
Training Dataset
(1036, 1024) (1036,)
Validation Dataset
(122, 1024) (122,)




Matthews Correlation :  0.4290639352898922
Confusion Matrix : 
 [[47 14]
 [21 40]]
Accuracy on test set:    0.7131147540983607
Sensitivity:    0.6557377049180327 	 Specificity:    0.7704918032786885
Area Under Curve:    0.7131147540983606
(520, 1024) (59, 1024)
(13650, 1024) (1940, 1024)
Training Dataset
(1040, 1024) (1040,)
Validation Dataset
(118, 1024) (118,)




Matthews Correlation :  0.49069974606294714
Confusion Matrix : 
 [[51  8]
 [23 36]]
Accuracy on test set:    0.7372881355932204
Sen

# XGBoost 10 Fold CV

In [3]:
# pip install xgboost

In [5]:
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler

a = random.sample(range(1, 1000000), 20)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Positive_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = XGBClassifier()

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("XGB Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("XGB Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(509, 1024) (68, 1024)
(14111, 1024) (1500, 1024)
Training Dataset
(1018, 1024) (1018,)
Validation Dataset
(136, 1024) (136,)




Matthews Correlation :  0.4714045207910317
Confusion Matrix : 
 [[48 20]
 [16 52]]
Accuracy on test set:    0.7352941176470589
Sensitivity:    0.7647058823529411 	 Specificity:    0.7058823529411765
Area Under Curve:    0.7352941176470587
(543, 1024) (34, 1024)
(14233, 1024) (1378, 1024)
Training Dataset
(1086, 1024) (1086,)
Validation Dataset
(68, 1024) (68,)




Matthews Correlation :  0.5303300858899106
Confusion Matrix : 
 [[25  9]
 [ 7 27]]
Accuracy on test set:    0.7647058823529411
Sensitivity:    0.7941176470588235 	 Specificity:    0.7352941176470589
Area Under Curve:    0.7647058823529411
(525, 1024) (52, 1024)
(14285, 1024) (1326, 1024)
Training Dataset
(1050, 1024) (1050,)
Validation Dataset
(104, 1024) (104,)




Matthews Correlation :  0.5971484496103001
Confusion Matrix : 
 [[40 12]
 [ 9 43]]
Accuracy on test set:    0.7980769230769231
Sensiti

Matthews Correlation :  0.6010407640085653
Confusion Matrix : 
 [[49  8]
 [15 42]]
Accuracy on test set:    0.7982456140350878
Sensitivity:    0.7368421052631579 	 Specificity:    0.8596491228070176
Area Under Curve:    0.7982456140350878
(388, 1024) (192, 1024)
(13990, 1024) (1781, 1024)
Training Dataset
(776, 1024) (776,)
Validation Dataset
(384, 1024) (384,)




Matthews Correlation :  0.3119336237158033
Confusion Matrix : 
 [[109  83]
 [ 50 142]]
Accuracy on test set:    0.6536458333333334
Sensitivity:    0.7395833333333334 	 Specificity:    0.5677083333333334
Area Under Curve:    0.6536458333333333
(515, 1024) (65, 1024)
(14276, 1024) (1495, 1024)
Training Dataset
(1030, 1024) (1030,)
Validation Dataset
(130, 1024) (130,)




Matthews Correlation :  0.5387166126295451
Confusion Matrix : 
 [[51 14]
 [16 49]]
Accuracy on test set:    0.7692307692307693
Sensitivity:    0.7538461538461538 	 Specificity:    0.7846153846153846
Area Under Curve:    0.7692307692307693
(541, 1024) (39, 102

Matthews Correlation :  0.475
Confusion Matrix : 
 [[59 21]
 [21 59]]
Accuracy on test set:    0.7375
Sensitivity:    0.7375 	 Specificity:    0.7375
Area Under Curve:    0.7375
(539, 1024) (35, 1024)
(14422, 1024) (1286, 1024)
Training Dataset
(1078, 1024) (1078,)
Validation Dataset
(70, 1024) (70,)




Matthews Correlation :  0.5196152422706631
Confusion Matrix : 
 [[29  6]
 [11 24]]
Accuracy on test set:    0.7571428571428571
Sensitivity:    0.6857142857142857 	 Specificity:    0.8285714285714286
Area Under Curve:    0.7571428571428571
(524, 1024) (50, 1024)
(12780, 1024) (2928, 1024)
Training Dataset
(1048, 1024) (1048,)
Validation Dataset
(100, 1024) (100,)




Matthews Correlation :  0.48989794855663565
Confusion Matrix : 
 [[42  8]
 [18 32]]
Accuracy on test set:    0.74
Sensitivity:    0.64 	 Specificity:    0.84
Area Under Curve:    0.7400000000000001
XGB Mean MCC = 0.5020 ± 0.0626 and Mean AUC = 0.7488 ± 0.0317
XGB Mean MCC:    0.502016129594884
Mean Sensitivity = 0.7250 ± 0.

Matthews Correlation :  0.2825419403198024
Confusion Matrix : 
 [[129  24]
 [ 90  63]]
Accuracy on test set:    0.6274509803921569
Sensitivity:    0.4117647058823529 	 Specificity:    0.8431372549019608
Area Under Curve:    0.6274509803921567
(514, 1024) (66, 1024)
(14048, 1024) (1634, 1024)
Training Dataset
(1028, 1024) (1028,)
Validation Dataset
(132, 1024) (132,)




Matthews Correlation :  0.5853873486290241
Confusion Matrix : 
 [[42 24]
 [ 5 61]]
Accuracy on test set:    0.7803030303030303
Sensitivity:    0.9242424242424242 	 Specificity:    0.6363636363636364
Area Under Curve:    0.7803030303030304
(514, 1024) (66, 1024)
(13593, 1024) (2089, 1024)
Training Dataset
(1028, 1024) (1028,)
Validation Dataset
(132, 1024) (132,)




Matthews Correlation :  0.4129483209670112
Confusion Matrix : 
 [[51 15]
 [24 42]]
Accuracy on test set:    0.7045454545454546
Sensitivity:    0.6363636363636364 	 Specificity:    0.7727272727272727
Area Under Curve:    0.7045454545454545
(536, 1024) (44, 10

(526, 1024) (53, 1024)
(13485, 1024) (2166, 1024)
Training Dataset
(1052, 1024) (1052,)
Validation Dataset
(106, 1024) (106,)




Matthews Correlation :  0.4890691894042119
Confusion Matrix : 
 [[46  7]
 [21 32]]
Accuracy on test set:    0.7358490566037735
Sensitivity:    0.6037735849056604 	 Specificity:    0.8679245283018868
Area Under Curve:    0.7358490566037735
(499, 1024) (80, 1024)
(13762, 1024) (1889, 1024)
Training Dataset
(998, 1024) (998,)
Validation Dataset
(160, 1024) (160,)




Matthews Correlation :  0.5886508336200406
Confusion Matrix : 
 [[66 14]
 [19 61]]
Accuracy on test set:    0.79375
Sensitivity:    0.7625 	 Specificity:    0.825
Area Under Curve:    0.79375
(532, 1024) (47, 1024)
(12628, 1024) (3023, 1024)
Training Dataset
(1064, 1024) (1064,)
Validation Dataset
(94, 1024) (94,)




Matthews Correlation :  0.5443794453866778
Confusion Matrix : 
 [[41  6]
 [16 31]]
Accuracy on test set:    0.7659574468085106
Sensitivity:    0.6595744680851063 	 Specificity:    0.8

Matthews Correlation :  0.18062322183437898
Confusion Matrix : 
 [[116  41]
 [ 89  68]]
Accuracy on test set:    0.5859872611464968
Sensitivity:    0.43312101910828027 	 Specificity:    0.7388535031847133
Area Under Curve:    0.5859872611464968
(485, 1024) (78, 1024)
(11558, 1024) (3581, 1024)
Training Dataset
(970, 1024) (970,)
Validation Dataset
(156, 1024) (156,)




Matthews Correlation :  0.4659217872346752
Confusion Matrix : 
 [[67 11]
 [32 46]]
Accuracy on test set:    0.7243589743589743
Sensitivity:    0.5897435897435898 	 Specificity:    0.8589743589743589
Area Under Curve:    0.7243589743589745
XGB Mean MCC = 0.4465 ± 0.1602 and Mean AUC = 0.7199 ± 0.0788
XGB Mean MCC:    0.4464801379735863
Mean Sensitivity = 0.7085 ± 0.1570 and Mean Specificity = 0.7313 ± 0.0559 and Mean Accuracy = 0.7199 ± 0.0788
Mean Precision = 0.7211 ± 0.0622 














(517, 1024) (56, 1024)
(13976, 1024) (1318, 1024)
Training Dataset
(1034, 1024) (1034,)
Validation Dataset
(112, 1024) (112,)




Ma

(538, 1024) (42, 1024)
(14208, 1024) (1557, 1024)
Training Dataset
(1076, 1024) (1076,)
Validation Dataset
(84, 1024) (84,)




Matthews Correlation :  0.5740378478819279
Confusion Matrix : 
 [[35  7]
 [11 31]]
Accuracy on test set:    0.7857142857142857
Sensitivity:    0.7380952380952381 	 Specificity:    0.8333333333333334
Area Under Curve:    0.7857142857142858
(527, 1024) (53, 1024)
(14701, 1024) (1064, 1024)
Training Dataset
(1054, 1024) (1054,)
Validation Dataset
(106, 1024) (106,)




Matthews Correlation :  0.5329708913388167
Confusion Matrix : 
 [[44  9]
 [16 37]]
Accuracy on test set:    0.7641509433962265
Sensitivity:    0.6981132075471698 	 Specificity:    0.8301886792452831
Area Under Curve:    0.7641509433962265
(527, 1024) (53, 1024)
(12273, 1024) (3492, 1024)
Training Dataset
(1054, 1024) (1054,)
Validation Dataset
(106, 1024) (106,)




Matthews Correlation :  0.5449492609130662
Confusion Matrix : 
 [[47  6]
 [19 34]]
Accuracy on test set:    0.7641509433962265
Sensiti

(517, 1024) (38, 1024)
(12317, 1024) (3210, 1024)
Training Dataset
(1034, 1024) (1034,)
Validation Dataset
(76, 1024) (76,)




Matthews Correlation :  0.6227523687795277
Confusion Matrix : 
 [[37  1]
 [15 23]]
Accuracy on test set:    0.7894736842105263
Sensitivity:    0.6052631578947368 	 Specificity:    0.9736842105263158
Area Under Curve:    0.7894736842105262
(499, 1024) (56, 1024)
(13651, 1024) (1876, 1024)
Training Dataset
(998, 1024) (998,)
Validation Dataset
(112, 1024) (112,)




Matthews Correlation :  0.5773502691896257
Confusion Matrix : 
 [[48  8]
 [16 40]]
Accuracy on test set:    0.7857142857142857
Sensitivity:    0.7142857142857143 	 Specificity:    0.8571428571428571
Area Under Curve:    0.7857142857142858
(487, 1024) (68, 1024)
(14264, 1024) (1263, 1024)
Training Dataset
(974, 1024) (974,)
Validation Dataset
(136, 1024) (136,)




Matthews Correlation :  0.7269837638573317
Confusion Matrix : 
 [[63  5]
 [14 54]]
Accuracy on test set:    0.8602941176470589
Sensitivity

(507, 1024) (64, 1024)
(13788, 1024) (1466, 1024)
Training Dataset
(1014, 1024) (1014,)
Validation Dataset
(128, 1024) (128,)




Matthews Correlation :  0.3281650616569468
Confusion Matrix : 
 [[43 21]
 [22 42]]
Accuracy on test set:    0.6640625
Sensitivity:    0.65625 	 Specificity:    0.671875
Area Under Curve:    0.6640625
(503, 1024) (68, 1024)
(14422, 1024) (832, 1024)
Training Dataset
(1006, 1024) (1006,)
Validation Dataset
(136, 1024) (136,)




Matthews Correlation :  0.4124789556921527
Confusion Matrix : 
 [[46 22]
 [18 50]]
Accuracy on test set:    0.7058823529411765
Sensitivity:    0.7352941176470589 	 Specificity:    0.6764705882352942
Area Under Curve:    0.7058823529411766
(525, 1024) (46, 1024)
(11960, 1024) (3294, 1024)
Training Dataset
(1050, 1024) (1050,)
Validation Dataset
(92, 1024) (92,)




Matthews Correlation :  0.6305838039516608
Confusion Matrix : 
 [[38  8]
 [ 9 37]]
Accuracy on test set:    0.8152173913043478
Sensitivity:    0.8043478260869565 	 Specificit

# SVM 10 Fold CV

In [11]:
from sklearn import svm
from imblearn.under_sampling import RandomUnderSampler

a = random.sample(range(1, 1000000), 1)

for i in range(len(a)):
    seed = a[i]
    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)
    windows = []
    start = 0
    stride = 15
    window_size = 15
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    MCC = []
    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Positive_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        print("Validation Dataset")
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
        print(xval.shape,yval.shape)

        print("\n\n\n")

        clf = svm.SVC(C=10, kernel='rbf')

        clf.fit(X_train, y_train)

        y_pred = np.array(clf.predict(xval))

        Accuracy_score = accuracy_score(yval, y_pred)

        print("Matthews Correlation : ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accuracy_score)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        acc.append(Accuracy_score)

        cvscores.append(mcc)

        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        PRE = TP/(TP+FP)

        precision.append(PRE)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)



        print("Area Under Curve:   ",roc_auc_test)
        auc_scores.append(roc_auc_test)   



    print("SVM Mean MCC = %.4f ± %.4f and Mean AUC = %.4f ± %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))
    MCC.append(np.mean(cvscores))

    print("SVM Mean MCC:   ",np.mean(MCC))
    print("Mean Sensitivity = %.4f ± %.4f and Mean Specificity = %.4f ± %.4f and Mean Accuracy = %.4f ± %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))


    print("Mean Precision = %.4f ± %.4f " % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

(544, 1024) (34, 1024)
(14203, 1024) (1066, 1024)
Training Dataset
(1088, 1024) (1088,)
Validation Dataset
(68, 1024) (68,)




Matthews Correlation :  0.5976143046671968
Confusion Matrix : 
 [[24 10]
 [ 4 30]]
Accuracy on test set:    0.7941176470588235
Sensitivity:    0.8823529411764706 	 Specificity:    0.7058823529411765
Area Under Curve:    0.7941176470588235
(547, 1024) (31, 1024)
(14127, 1024) (1142, 1024)
Training Dataset
(1094, 1024) (1094,)
Validation Dataset
(62, 1024) (62,)




Matthews Correlation :  0.5495319562599505
Confusion Matrix : 
 [[25  6]
 [ 8 23]]
Accuracy on test set:    0.7741935483870968
Sensitivity:    0.7419354838709677 	 Specificity:    0.8064516129032258
Area Under Curve:    0.7741935483870968
(532, 1024) (46, 1024)
(12462, 1024) (2807, 1024)
Training Dataset
(1064, 1024) (1064,)
Validation Dataset
(92, 1024) (92,)




Matthews Correlation :  0.3977220302613713
Confusion Matrix : 
 [[40  6]
 [23 23]]
Accuracy on test set:    0.6847826086956522
Sensitivity

# Thank You